In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 

import warnings
warnings.filterwarnings("ignore")

from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.preprocessing import LabelEncoder 

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier




In [4]:
# load the datasets

df1 = pd.read_excel('case_study1.xlsx')

df2 = pd.read_excel('case_study2.xlsx')

In [5]:
print(df1.shape)
print(df2.shape)

(51336, 26)
(51336, 62)


In [6]:
# remove nulls

df1 = df1[df1['Age_Newest_TL'] != -99999]

df1.shape

(51296, 26)

In [7]:
# columns with more than 10k null values 

cols_removed = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        cols_removed.append(i)

cols_removed

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [8]:
# remove cols with null values

df2 = df2.drop(cols_removed, axis=1)

df2.shape

(51336, 54)

In [9]:
# remove null rows

for i in df2.columns:
    df2 = df2.loc[df2[i] != -99999]

df2.shape
    

(42066, 54)

In [10]:
# checking commom columns

for i in list(df1.columns):
    if i in list(df2.columns):
        print(i)

PROSPECTID


In [11]:
# join both datasets

df = df1.merge(df2, how='inner', on='PROSPECTID')

df.shape

(42064, 79)

In [12]:
# check categorical columns

for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [13]:
print(df['MARITALSTATUS'].unique())
print(df['EDUCATION'].unique())
print(df['GENDER'].unique())
print(df['last_prod_enq2'].unique())
print(df['first_prod_enq2'].unique())
print(df['Approved_Flag'].unique())

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']
['P2' 'P1' 'P3' 'P4']


In [14]:
# chi-square test 

for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, "---", pval)

MARITALSTATUS --- 3.5781808610388605e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.9079361001865664e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.849976105554191e-287


In [15]:
# all values less than equal to 0.05, will accept all 
# associated with target variable 

In [16]:
# check numerical columns

num_cols = []

for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID', 'Approved_Flag']:
        num_cols.append(i)

num_cols

['Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'pct_active_tl',
 'pct_closed_tl',
 'Total_TL_opened_L12M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'Auto_TL',
 'CC_TL',
 'Consumer_TL',
 'Gold_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',

In [17]:
# VIF sequentially check

vif_data = df[num_cols]
total_cols = vif_data.shape[1]
cols_to_keep = []
col_index = 0

for i in range(0, total_cols):
    vif_value = variance_inflation_factor(vif_data, col_index)
    print(col_index, "---", vif_value)

    if vif_value <= 6:
        cols_to_keep.append(num_cols[i])
        col_index = col_index + 1

    else: 
        vif_data = vif_data.drop(num_cols[i], axis=1)


0 --- inf
0 --- inf
0 --- 11.320180023967982
0 --- 8.36369803500036
0 --- 6.5206478777909425
0 --- 5.14950161821261
1 --- 2.611111040579735
2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477212
2 --- 3.832800792153082
3 --- 6.0996533816466405
3 --- 5.581352009642814
4 --- 1.9855843530987702
5 --- inf
5 --- 4.809538302819332
6 --- 23.270628983464636
6 --- 30.595522588099946
6 --- 4.384346405965575
7 --- 3.0646584155234122
8 --- 2.898639771299225
9 --- 4.377876915347337
10 --- 2.2078535836958486
11 --- 4.916914200506877
12 --- 5.214702030064743
13 --- 3.3861625024231516
14 --- 7.84058330947899
14 --- 5.255034641721459
15 --- inf
15 --- 7.380634506427207
15 --- 1.421005001517572
16 --- 8.083255010190301
16 --- 1.6241227524040012
17 --- 7.257811920140015
17 --- 15.596243832683006
17 --- 1.825857047132431
18 --- 1.5080839450032724
19 --- 2.1720888348245815
20 --- 2.6233975535272367
21 --- 2.2959970812106216
22 --- 7.360578319196457
22 --- 2.1602387773102514
23 --- 2.8686288267891493
24

In [18]:
len(cols_to_keep)

39

In [19]:
# check anova

cols_to_keep_num = []

for i in cols_to_keep:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']

    f_statistics, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        cols_to_keep_num.append(i)


len(cols_to_keep_num)

37

In [39]:
# listing all the final features

features = cols_to_keep_num + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]

In [40]:
# ordinal encoding for education

df.loc[df['EDUCATION']=='SSC', ['EDUCATION']] = 1
df.loc[df['EDUCATION']=='12TH', ['EDUCATION']] = 2
df.loc[df['EDUCATION']=='UNDER GRADUATE', ['EDUCATION']] = 3
df.loc[df['EDUCATION']=='GRADUATE', ['EDUCATION']] = 3
df.loc[df['EDUCATION']=='POST-GRADUATE', ['EDUCATION']] = 4
df.loc[df['EDUCATION']=='PROFESSIONAL', ['EDUCATION']] = 3
df.loc[df['EDUCATION']=='OTHERS', ['EDUCATION']] = 1

df['EDUCATION'] = df['EDUCATION'].astype(int)
df['EDUCATION'].value_counts()

EDUCATION
3    18931
2    11703
1     9532
4     1898
Name: count, dtype: int64

In [41]:
# one-hot encoding for other categorical columns

df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'], drop_first=True)
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 51 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [26]:
# random forest

x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

x_trian, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)

rf_classifier.fit(x_trian, y_train)

y_pred = rf_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print()
print(f'Accuracy = {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f'Class {v}')
    print(f'Precision = {precision[i]}')
    print(f'Recall = {recall[i]}')
    print(f'F1_score = {f1_score[i]}')
    print()



Accuracy = 0.7663140377986449

Class p1
Precision = 0.8411214953271028
Recall = 0.7100591715976331
F1_score = 0.7700534759358288

Class p2
Precision = 0.7974855589534489
Recall = 0.930426164519326
F1_score = 0.8588418259994511

Class p3
Precision = 0.4390625
Recall = 0.2120754716981132
F1_score = 0.2860050890585242

Class p4
Precision = 0.7293889427740058
Recall = 0.7308066083576288
F1_score = 0.7300970873786408



In [27]:
# decision tree

x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

x_trian, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)

dt_model.fit(x_trian, y_train)

y_pred = dt_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print()
print(f'Accuracy = {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f'Class {v}')
    print(f'Precision = {precision[i]}')
    print(f'Recall = {recall[i]}')
    print(f'F1_score = {f1_score[i]}')
    print()


Accuracy = 0.707714251753239

Class p1
Precision = 0.7158098933074685
Recall = 0.727810650887574
F1_score = 0.721760391198044

Class p2
Precision = 0.8072805139186295
Recall = 0.822001982160555
F1_score = 0.8145747397367904

Class p3
Precision = 0.34269662921348315
Recall = 0.3222641509433962
F1_score = 0.33216647218980944

Class p4
Precision = 0.6426426426426426
Recall = 0.6239067055393586
F1_score = 0.6331360946745562



In [28]:
# xgboost

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_trian, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

xgb_classifier.fit(x_trian, y_train)

y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print()
print(f'Accuracy = {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f'Class {v}')
    print(f'Precision = {precision[i]}')
    print(f'Recall = {recall[i]}')
    print(f'F1_score = {f1_score[i]}')
    print()


Accuracy = 0.7726138119576845

Class p1
Precision = 0.8232804232804233
Recall = 0.7672583826429981
F1_score = 0.7942827973455845

Class p2
Precision = 0.8261887608069164
Recall = 0.9092170465807731
F1_score = 0.8657167122770596

Class p3
Precision = 0.44532130777903045
Recall = 0.2981132075471698
F1_score = 0.35714285714285715

Class p4
Precision = 0.7191448007774538
Recall = 0.7191448007774538
F1_score = 0.7191448007774538



In [34]:
import optuna
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# -------------------------
# Prepare data ONCE
# -------------------------
y = df_encoded["Approved_Flag"]
X = df_encoded.drop(columns=["Approved_Flag"])

le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc,
    test_size=0.2,
    random_state=42,
    stratify=y_enc
)

# -------------------------
# Optuna objective
# -------------------------
def objective(trial):

    params = {
        "objective": "multi:softmax",
        "num_class": 4,
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 0.9, step=0.2),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.001, 0.01, 0.1, 1.0]),
        "max_depth": trial.suggest_categorical("max_depth", [3, 5, 8, 10]),
        "reg_alpha": trial.suggest_categorical("alpha", [1, 10, 100]),
        "n_estimators": trial.suggest_categorical("n_estimators", [10, 50, 100]),
        "random_state": 42,
        "n_jobs": -1,
        "eval_metric": "mlogloss"
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    test_preds  = model.predict(X_test)

    train_acc = accuracy_score(y_train, train_preds)
    test_acc  = accuracy_score(y_test, test_preds)

    # store train accuracy for the best trial
    trial.set_user_attr("train_accuracy", train_acc)

    return test_acc

# -------------------------
# Run optimization
# -------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, show_progress_bar=False)

# -------------------------
# Print ONLY best result
# -------------------------
best_trial = study.best_trial

print("Best Test Accuracy :", best_trial.value)
print("Best Train Accuracy:", best_trial.user_attrs["train_accuracy"])
print("Best Parameters    :", best_trial.params)


[I 2026-01-26 23:20:44,889] A new study created in memory with name: no-name-b11bb279-206d-4525-8e66-a9637f3e0828
[I 2026-01-26 23:20:45,094] Trial 0 finished with value: 0.7547842624509687 and parameters: {'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'max_depth': 10, 'alpha': 10, 'n_estimators': 10}. Best is trial 0 with value: 0.7547842624509687.
[I 2026-01-26 23:20:45,253] Trial 1 finished with value: 0.7102103886841793 and parameters: {'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'max_depth': 3, 'alpha': 1, 'n_estimators': 10}. Best is trial 0 with value: 0.7547842624509687.
[I 2026-01-26 23:20:46,377] Trial 2 finished with value: 0.7677404017591822 and parameters: {'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'max_depth': 5, 'alpha': 100, 'n_estimators': 100}. Best is trial 2 with value: 0.7677404017591822.
[I 2026-01-26 23:20:46,618] Trial 3 finished with value: 0.605016046594556 and parameters: {'colsample_bytree': 0.70000000000

Best Test Accuracy : 0.7796267680969927
Best Train Accuracy: 0.8056521351520014
Best Parameters    : {'colsample_bytree': 0.5, 'learning_rate': 1.0, 'max_depth': 3, 'alpha': 1, 'n_estimators': 50}


In [42]:
pd.set_option('display.max_columns', None)

df_encoded.describe()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION
count,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.00000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,4.206400e+04,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000
mean,0.179032,0.097783,0.825504,0.160365,0.525746,0.145921,0.076241,0.328000,2.921334,2.341646,1.116489,46.498074,13.970046,218.601607,14.314758,0.336963,0.438879,3.279978,0.063831,0.002211,0.009224,0.02451,0.004279,0.016713,11.803918,0.268924,0.779194,264.854507,1.230458,2.692990e+04,110.345783,0.102962,0.193063,0.195497,0.064186,0.252235,0.056580,2.313689
std,0.278043,0.210957,1.537208,0.258831,1.106442,0.549314,0.358582,0.916368,6.379764,3.405397,2.486801,42.109230,18.835191,422.282417,54.056303,1.097356,2.148400,7.566312,0.799989,0.081704,0.220786,0.62189,0.184461,0.573762,46.422091,1.019459,1.802092,466.585002,2.069461,2.084300e+04,75.629967,0.303913,0.394707,0.367414,0.225989,0.434300,0.231042,0.871070
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,4.000000,51.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,1.800000e+04,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,34.000000,7.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,79.000000,1.000000,2.400000e+04,92.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,0.333000,0.100000,1.000000,0.250000,1.000000,0.000000,0.000000,0.000000,3.000000,3.000000,1.000000,65.000000,16.000000,146.000000,15.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,11.000000,0.000000,1.000000,302.000000,2.000000,3.100000e+04,131.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000
max,1.000000,1.000000,33.000000,1.000000,34.000000,27.000000,10.000000,29.000000,235.000000,55.000000,80.000000,385.000000,359.000000,6065.000000,900.000000,20.000000,52.000000,122.000000,41.000000,5.000000,12.000000,35.00000,12.000000,72.000000,900.000000,24.000000,44.000000,4768.000000,42.000000,2.500000e+06,1020.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000


In [43]:
# Apply standard scaler 

from sklearn.preprocessing import StandardScaler

columns_to_be_scaled = ['Age_Oldest_TL','Age_Newest_TL','time_since_recent_payment',
'max_recent_level_of_deliq','recent_level_of_deliq',
'time_since_recent_enq','NETMONTHLYINCOME','Time_With_Curr_Empr']

for i in columns_to_be_scaled:
    column_data = df_encoded[i].values.reshape(-1, 1)
    scaler = StandardScaler()
    scaled_column = scaler.fit_transform(column_data)
    df_encoded[i] = scaled_column


In [45]:
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_trian, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

xgb_classifier.fit(x_trian, y_train)

y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print()
print(f'Accuracy = {accuracy}')
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f'Class {v}')
    print(f'Precision = {precision[i]}')
    print(f'Recall = {recall[i]}')
    print(f'F1_score = {f1_score[i]}')
    print()


Accuracy = 0.7726138119576845

Class p1
Precision = 0.8232804232804233
Recall = 0.7672583826429981
F1_score = 0.7942827973455845

Class p2
Precision = 0.8261887608069164
Recall = 0.9092170465807731
F1_score = 0.8657167122770596

Class p3
Precision = 0.44532130777903045
Recall = 0.2981132075471698
F1_score = 0.35714285714285715

Class p4
Precision = 0.7191448007774538
Recall = 0.7191448007774538
F1_score = 0.7191448007774538



In [46]:
y = df_encoded["Approved_Flag"]
X = df_encoded.drop(columns=["Approved_Flag"])

le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc,
    test_size=0.2,
    random_state=42,
    stratify=y_enc
)

# -------------------------
# Optuna objective
# -------------------------
def objective(trial):

    params = {
        "objective": "multi:softmax",
        "num_class": 4,
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 0.9, step=0.2),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.001, 0.01, 0.1, 1.0]),
        "max_depth": trial.suggest_categorical("max_depth", [3, 5, 8, 10]),
        "reg_alpha": trial.suggest_categorical("alpha", [1, 10, 100]),
        "n_estimators": trial.suggest_categorical("n_estimators", [10, 50, 100]),
        "random_state": 42,
        "n_jobs": -1,
        "eval_metric": "mlogloss"
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    test_preds  = model.predict(X_test)

    train_acc = accuracy_score(y_train, train_preds)
    test_acc  = accuracy_score(y_test, test_preds)

    # store train accuracy for the best trial
    trial.set_user_attr("train_accuracy", train_acc)

    return test_acc

# -------------------------
# Run optimization
# -------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, show_progress_bar=False)

# -------------------------
# Print ONLY best result
# -------------------------
best_trial = study.best_trial

print("Best Test Accuracy :", best_trial.value)
print("Best Train Accuracy:", best_trial.user_attrs["train_accuracy"])
print("Best Parameters    :", best_trial.params)


[I 2026-01-26 23:33:09,613] A new study created in memory with name: no-name-d709662a-fc5d-495c-9cbe-465224c68fa5
[I 2026-01-26 23:33:14,768] Trial 0 finished with value: 0.605016046594556 and parameters: {'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.001, 'max_depth': 10, 'alpha': 1, 'n_estimators': 100}. Best is trial 0 with value: 0.605016046594556.
[I 2026-01-26 23:33:14,934] Trial 1 finished with value: 0.6900035659099013 and parameters: {'colsample_bytree': 0.1, 'learning_rate': 1.0, 'max_depth': 3, 'alpha': 100, 'n_estimators': 10}. Best is trial 1 with value: 0.6900035659099013.
[I 2026-01-26 23:33:15,283] Trial 2 finished with value: 0.7719006299774159 and parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 8, 'alpha': 1, 'n_estimators': 10}. Best is trial 2 with value: 0.7719006299774159.
[I 2026-01-26 23:33:15,815] Trial 3 finished with value: 0.7566860810650184 and parameters: {'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, '

Best Test Accuracy : 0.7796267680969927
Best Train Accuracy: 0.8056521351520014
Best Parameters    : {'colsample_bytree': 0.5, 'learning_rate': 1.0, 'max_depth': 3, 'alpha': 1, 'n_estimators': 50}


In [81]:
# predict for unseen data

un = pd.read_excel('/Users/shraddhagupta/Downloads/Unseen_Dataset.xlsx')
un.shape


(100, 42)

In [82]:
# ordinal encoding for education

un.loc[un['EDUCATION']=='SSC', ['EDUCATION']] = 1
un.loc[un['EDUCATION']=='12TH', ['EDUCATION']] = 2
un.loc[un['EDUCATION']=='UNDER GRADUATE', ['EDUCATION']] = 3
un.loc[un['EDUCATION']=='GRADUATE', ['EDUCATION']] = 3
un.loc[un['EDUCATION']=='POST-GRADUATE', ['EDUCATION']] = 4
un.loc[un['EDUCATION']=='PROFESSIONAL', ['EDUCATION']] = 3
un.loc[un['EDUCATION']=='OTHERS', ['EDUCATION']] = 1

un['EDUCATION'] = un['EDUCATION'].astype(int)


# one-hot encoding for other categorical columns

df_unseen = pd.get_dummies(un, columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'], drop_first=True)
df_unseen.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 50 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               100 non-null    float64
 1   pct_tl_closed_L6M             100 non-null    float64
 2   Tot_TL_closed_L12M            100 non-null    int64  
 3   pct_tl_closed_L12M            100 non-null    float64
 4   Tot_Missed_Pmnt               100 non-null    int64  
 5   CC_TL                         100 non-null    int64  
 6   Home_TL                       100 non-null    int64  
 7   PL_TL                         100 non-null    int64  
 8   Secured_TL                    100 non-null    int64  
 9   Unsecured_TL                  100 non-null    int64  
 10  Other_TL                      100 non-null    int64  
 11  Age_Oldest_TL                 100 non-null    int64  
 12  Age_Newest_TL                 100 non-null    int64  
 13  time_s

In [83]:
model = xgb.XGBClassifier(objective='multi:softmax', num_class=4, 
                         colsample_bytree=0.9, learning_rate=1,
                         max_depth=3, alpha=1, n_estimators=50)

x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_trian, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

model.fit(x_trian, y_train)

y_pred_unseen = model.predict(df_unseen)

df_unseen['Target'] = y_pred_unseen


In [84]:
y_pred = label_encoder.inverse_transform(y_pred_unseen)
df_unseen['Target_v'] = y_pred

df_unseen['Target_v'].value_counts()

Target_v
P1    74
P2    20
P3     6
Name: count, dtype: int64

In [32]:
# # Define the hyperparameter grid
# param_grid = {
#   'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
#   'learning_rate'   : [0.001, 0.01, 0.1, 1],
#   'max_depth'       : [3, 5, 8, 10],
#   'alpha'           : [1, 10, 100],
#   'n_estimators'    : [10,50,100]
# }

# index = 0

# answers_grid = {
#     'combination'       :[],
#     'train_Accuracy'    :[],
#     'test_Accuracy'     :[],
#     'colsample_bytree'  :[],
#     'learning_rate'     :[],
#     'max_depth'         :[],
#     'alpha'             :[],
#     'n_estimators'      :[]

#     }


# # Loop through each combination of hyperparameters
# for colsample_bytree in param_grid['colsample_bytree']:
#   for learning_rate in param_grid['learning_rate']:
#     for max_depth in param_grid['max_depth']:
#       for alpha in param_grid['alpha']:
#           for n_estimators in param_grid['n_estimators']:
             
#               index = index + 1
             
#               # Define and train the XGBoost model
#               model = xgb.XGBClassifier(objective='multi:softmax',  
#                                        num_class=4,
#                                        colsample_bytree = colsample_bytree,
#                                        learning_rate = learning_rate,
#                                        max_depth = max_depth,
#                                        alpha = alpha,
#                                        n_estimators = n_estimators)
               
       
                     
#               y = df_encoded['Approved_Flag']
#               x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

#               label_encoder = LabelEncoder()
#               y_encoded = label_encoder.fit_transform(y)


#               x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)


#               model.fit(x_train, y_train)
  

       
#               # Predict on training and testing sets
#               y_pred_train = model.predict(x_train)
#               y_pred_test = model.predict(x_test)
       
       
#               # Calculate train and test results
              
#               train_accuracy =  accuracy_score (y_train, y_pred_train)
#               test_accuracy  =  accuracy_score (y_test , y_pred_test)
              
              
       
#               # Include into the lists
#               answers_grid ['combination']   .append(index)
#               answers_grid ['train_Accuracy']    .append(train_accuracy)
#               answers_grid ['test_Accuracy']     .append(test_accuracy)
#               answers_grid ['colsample_bytree']   .append(colsample_bytree)
#               answers_grid ['learning_rate']      .append(learning_rate)
#               answers_grid ['max_depth']          .append(max_depth)
#               answers_grid ['alpha']              .append(alpha)
#               answers_grid ['n_estimators']       .append(n_estimators)
       
       
#               # Print results for this combination
#               print(f"Combination {index}")
#               print(f"colsample_bytree: {colsample_bytree}, learning_rate: {learning_rate}, max_depth: {max_depth}, alpha: {alpha}, n_estimators: {n_estimators}")
#               print(f"Train Accuracy: {train_accuracy:.2f}")
#               print(f"Test Accuracy : {test_accuracy :.2f}")
#               print("-" * 30)



In [33]:
# # Convert answers_grid into a DataFrame
# answers_df = pd.DataFrame(answers_grid)

# # Find the row with the best (highest) test accuracy
# best_row = answers_df.loc[answers_df['test_Accuracy'].idxmax()]

# print("✅ Best Hyperparameter Combination:")
# print(f"Combination #: {int(best_row['combination'])}")
# print(f"Train Accuracy: {best_row['train_Accuracy']:.4f}")
# print(f"Test Accuracy : {best_row['test_Accuracy']:.4f}")
# print(f"colsample_bytree: {best_row['colsample_bytree']}")
# print(f"learning_rate  : {best_row['learning_rate']}")
# print(f"max_depth      : {int(best_row['max_depth'])}")
# print(f"alpha          : {best_row['alpha']}")
# print(f"n_estimators   : {int(best_row['n_estimators'])}")

